<center><h1><b><span style="color:blue">Files, data, event processing</span></b></h1></center>

#### **Quick intro to the following packages**
- `uproot` - ROOT I/O in pure Python and NumPy.
- `awkward-array` - manipulate arrays of complex data structures as easily as NumPy.

&nbsp;
<center>
    <img src="images/logo_uproot.png" style="width:150px;"/>
    <h2><b><span style="color:green">ROOT I/O in pure Python and NumPy</span></b></h2>
</center>

### **What is ``uproot``?**

<span style="color:green">**Effectively what connects HEP data (ROOT format) with the Python scientific ecosystem around NumPy!**</span>

`uproot` provides very fast, efficient, and convenient access to ROOT trees.

- Pure Python + NumPy implementation of ROOT I/O.
- An array-centric view of ROOT TTree data:
  - Branches of simple types are simple arrays.
  - Branches of complex types are “jagged arrays”.
-  High performance for large baskets, despite Python’s slowness (because all per-entry operations are performed in NumPy).
- Greatest benefits: simplicity, minimal installation, set-up, and affinity with machine learning interfaces.

### **Why does it exist?**
1. To extract columnar data (branches) from a ROOT file without invoking the
event-handling infrastructure of the ROOT framework.
3. To express the semantics and conventions of the ROOT file format independently
of ROOT, in lieu of a formal specification.

### **Why Python + NumPy?**
- As stressed several times, the scientific Python ecosystem, including much of ML, is designed around a fundamental abstraction called the NumPy array.
- Working with computer scientists is easier when you can say, "*pip install uproot*".
- Implemented correctly, Python + NumPy doesn't have to be slow.
  - Finding the columnar data in a ROOT file may be done in slow Python, as long as
decompression and array manipulations are done by compiled code, see the now-old-ish performance study below.

<center><img src="images/Scikit-HEP_uproot_performance.png" width = "60%"/></center>

### **1. Getting data from a simple ROOT TTree**

By "simple" we mean a file without *jagged structures*, or nested structures with branch sizes depending on an event-by-event basis.

In [ ]:
import uproot

In [ ]:
f = uproot.open('data/Zmumu.root')

f

ROOT files, directories, and trees are like Python dicts with keys() and values().

In [ ]:
f.keys()

In [ ]:
t = f[b'events']
t.keys()

In [ ]:
t['M']

Uproot's main purpose is to read branches from ROOT files as NumPy arrays:

In [ ]:
t['M'].array()

All branches can be looked at with `t.arrays()`. A subset is specified e.g. as `t.arrays(['Run', 'Event'])`:

In [ ]:
t.arrays()

One can now start performing calculations such as the below. But this is not necessary the best way, think NumPy type of operations ;-).

In [ ]:
import numpy

for px,py,pz in t.iterate(["px1","py1","pz1"], outputtype=tuple):
    pt = numpy.sqrt(px**2 + py**2)
    eta = numpy.arctanh(pz / numpy.sqrt(px**2 + py**2 + pz**2))
    phi = numpy.arctan2(py, px)
    print(pt)
    print(eta)
    print(phi)

**Create a Pandas DataFrame**

In [ ]:
import pandas

df = t.pandas.df()
df

In [ ]:
t.pandas.df(['Run', 'Event', 'pt1', 'pt2']).head()

### **2. Data with jagged structure**

In [ ]:
f = uproot.open('data/uproot-tutorial-file.root')

f.keys()

In [ ]:
branches = f[b'Events'].arrays(namedecode='utf-8')
                               
branches

The jagged structure here comes from the number of muons per event, which is variable:

In [ ]:
branches['nMuon']

This becomes evident when checking for example the $p_T$ of all muons:

In [ ]:
branches['Muon_pt']

Print the $p_T$ for the muons in the first 10 events to trivially see the jagged structure:

In [ ]:
print(' \n'.join([str(elm) for elm in branches['Muon_pt'][:10]]))

We will get back to jagged arrays in a sec. Let's first show that `uproot` also has (limited) writing functionality.

### **3. Interoperability - writing files with uproot**

`uproot` version 3 started support for writing files!

In [ ]:
f = uproot.recreate("tmp.root")

In [ ]:
f["name"] = numpy.histogram(numpy.random.normal(0, 1, 100000), bins=20)

In [ ]:
f.closed

In [ ]:
f.keys()

In [ ]:
ar = f["name"]

ar.show()

But ... can the file actually be read back in ROOT ...? (Yes it can!)

If you have ROOT installed, try
```python
import ROOT

f = ROOT.TFile("tmp.root")
h = f.Get("name")

c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)
h.Draw('hist')
c.Draw()
```

If not, and you aren't working in Windows, try to install ROOT via Conda,
```python
!conda install root -c conda-forge
```
and then try the above.

&nbsp;<br><center><img src="images/logo_awkward-array.png" style="width: 150px;"/></center>

<center><h2><b><span style="color:green">Manipulate arrays of complex data structures as easily as NumPy</span></b></h2></center>

In [ ]:
import awkward

In [ ]:
branches

In [ ]:
table = awkward.Table(branches)

table

In [ ]:
table.nMuon[0]

In [ ]:
branches['nMuon'][0]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(table.nMuon, bins=10, range=(0, 10))
plt.xlabel('Number of muons in event')
plt.ylabel('Number of events');

How many muon entries are there in total?

In [ ]:
branches['nMuon'].sum()

In [ ]:
len(branches['Muon_pt']), len(branches['Muon_pt'].flatten())  # 235286 muons in 1e5 events

Plot the $p_T$ and $\eta$ of all muons:

In [ ]:
plt.hist(table.Muon_pt.flatten(), bins=100, range=(0, 100))
plt.xlabel('Muon pT')
plt.ylabel('Number of candidates')
plt.yscale('log');

In [ ]:
plt.hist(table.Muon_eta.flatten(), bins=100, range=(-2.5, 2.5))
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of candidates');

In [ ]:
%%timeit
len(branches['Muon_pt'].flatten())

In [ ]:
%%timeit
branches['nMuon'].sum()

Selections are done via masks. Let's create one that singles out events with a single muon:

In [ ]:
branches['nMuon'] == 1

In [ ]:
single_muon_mask = branches['nMuon'] == 1
single_muon_mask.sum()

Just checking:

In [ ]:
len(branches['Muon_pt'][single_muon_mask]), branches['Muon_pt'][single_muon_mask]

In [ ]:
plt.hist(branches['Muon_pt'][single_muon_mask].flatten(), bins=100, range=(0, 100))
plt.xlabel('Muon $p_{\mathrm{T}}$ [MeV]')
plt.ylabel('Number of single muons / 1 MeV')
plt.yscale('log')
plt.show()

Mask to selection muons within $|\eta| <2$:

In [ ]:
eta_mask = abs(branches['Muon_eta']) < 2
eta_mask

In [ ]:
eta_mask.flatten().sum()

In [ ]:
eta_mask.sum()

In [ ]:
eta_mask.sum().sum()

In [ ]:
plt.hist(branches['Muon_eta'].flatten(), bins=50, range=(-2.5, 2.5))
plt.title('No selection')
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of muons')
plt.show()

plt.hist(branches['Muon_eta'][eta_mask].flatten(), bins=50, range=(-2.5, 2.5))
plt.title('With $|\eta| < 2$ selection')
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of muons')
plt.show()

In [ ]:
len(single_muon_mask & eta_mask)

In [ ]:
plt.hist([branches['Muon_pt'][single_muon_mask & eta_mask].flatten(),
          branches['Muon_pt'][single_muon_mask & ~eta_mask].flatten()],
         label=['$|\eta| < 2$', '$|\eta| \geq 2$'],
         density=True,
         bins=25, range=(0, 50))
plt.xlabel('Muon $p_{\mathrm{T}}$ [GeV]')
plt.ylabel('Number of single muons / 2 GeV')
plt.legend()
plt.show()

Concentrate on 2-muon events:

In [ ]:
two_muons_mask = branches['nMuon'] == 2
two_muons_table = table[two_muons_mask]

In [ ]:
# Better methods to deal with Lorentz vectors are soon to be available

import uproot_methods

two_muons_p4 = uproot_methods.TLorentzVectorArray.from_ptetaphim(two_muons_table['Muon_pt'],
                                                                 two_muons_table['Muon_eta'],
                                                                 two_muons_table['Muon_phi'],
                                                                 two_muons_table['Muon_mass'])
two_muons_p4

In [ ]:
len(two_muons_p4)

In [ ]:
first_muon_p4 = two_muons_p4[:, 0]
second_muon_p4 = two_muons_p4[:, 1]

In [ ]:
first_muon_p4.delta_r(second_muon_p4)

In [ ]:
plt.hist(first_muon_p4.delta_r(second_muon_p4), bins=100)
plt.xlabel('$\Delta R$ between muons')
plt.ylabel('Number of two-muon events')
plt.show()

Further refine to $\mu^+\mu^-$ pairs - you see where're getting ;-):

In [ ]:
sum_p4 = first_muon_p4 + second_muon_p4
opposite_sign_muons_mask = two_muons_table['Muon_charge'][:, 0] != two_muons_table['Muon_charge'][:, 1]
dimuon_p4 = sum_p4[opposite_sign_muons_mask]
dimuon_p4

In [ ]:
import numpy as np

figsize_l, figsize_h = plt.rcParams["figure.figsize"]
plt.figure(figsize=(figsize_l*2.5, figsize_h*3.))

(yvals, binedges, patches) = plt.hist(dimuon_p4.mass, bins=np.logspace(np.log10(0.1), np.log10(1000), 200))

plt.xlabel('Dimuon invariant mass [GeV]')
plt.ylabel('Number of dimuon events')
plt.xscale('log')
plt.yscale('log')

import particle.particle.literals as lpart
from hepunits import GeV
    
list_particles = [getattr(lpart,name) for name in ('eta', 'rho_770_0', 'omega_782','phi_1020','Jpsi_1S', 'Z_0')]

for p in list_particles:
    binnumber = np.searchsorted(binedges, p.mass/GeV)
    plt.text(p.mass/GeV, yvals[binnumber-1]*1.02, '${}$'.format(p.latex_name), horizontalalignment='center', fontsize=16)

plt.show()

&nbsp;
<div class="alert alert-warning">

<b>Important note:</b>

These 2 packages are currently under a major overhaul!
</div>

- `awkward-array` is being replaced by an improved package nicknamed `awkward-1.0` for now. Developments are happening at [this GitHub repository](https://github.com/scikit-hep/awkward-1.0).
- `uproot` version 4 will be the upgrade version of `uproot` version 3 built atop `awkward-1.0`, see https://github.com/scikit-hep/uproot4.